In [1]:
import cufflinks
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment',None)
cufflinks.set_config_file(offline = True)

In [2]:
class FinancialData:
    url = 'http://hilpisch.com/pyalgo_eikon_eod_data.csv'
    def __init__(self, symbol):
        self.symbol = symbol
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col = 0, parse_dates =True).dropna()
    def prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data['returns'] = np.log(self.data/self.data.shift(1))
    def plot_data(self, cols = None):
        if cols is None:
            cols = [self.symbol]
        self.data[cols].plot(figsize = (15,6), title = self.symbol)
        
    

In [3]:
fd = FinancialData('EUR=')

In [5]:
import time

In [8]:
for bar in range(10):
    print(bar,fd.data.index[bar],fd.data[fd.symbol].iloc[bar])
    time.sleep(1)

0 2010-01-04 00:00:00 1.4411
1 2010-01-05 00:00:00 1.4368
2 2010-01-06 00:00:00 1.4412
3 2010-01-07 00:00:00 1.4318
4 2010-01-08 00:00:00 1.4412
5 2010-01-11 00:00:00 1.4513
6 2010-01-12 00:00:00 1.4494
7 2010-01-13 00:00:00 1.4509999999999998
8 2010-01-14 00:00:00 1.4502
9 2010-01-15 00:00:00 1.4382


In [178]:
class BacktestBase(FinancialData):
    def __init__(self,symbol,amount):
        super().__init__(symbol)
        self.initial_amount = amount
        self.current_balance = amount
        self.units = 0
        self.trades = 0
    def get_date_price(self,bar):
        date = str(self.data.index[bar])[:10]
        price = self.data[self.symbol].iloc[bar]
        return date,price
    def print_current_balance(self,bar):
        date,price = self.get_date_price(bar)
        print(f'{date} | current balance = {self.current_balance:.2f}')
    def print_net_wealth(self,bar):
        date,price = self.get_date_price(bar)
        net_wealth = self.current_balance + self.units*price
        print(f'{date} | net wealth = {net_wealth:.2f}')
    def place_buy_order(self,bar,units = None,amount = None):
        date,price = self.get_date_price(bar)
        if units is None:
            units = int(amount/price)
        self.current_balance -= units*price
        self.units += units
        self.trades += 1
        print(f'{date} | bought {units} at price {price}')
    def place_sell_order(self,bar,units = None,amount = None):
        date,price = self.get_date_price(bar)
        if units is None:
            units = int(amount/price)
        self.current_balance += units*price
        self.units -= units
        self.trades += 1
        print(f'{date} | sold {units} at price {price}')
    def close_out(self,bar):
        date,price = self.get_date_price(bar)
        print(55* '=')
        print(f'{date} | CLOSING OUT POSITION')
        print(55* '=')
        self.current_balance += self.units*price 
        perf = (self.current_balance-self.initial_amount)/self.initial_amount        
        print(f'{date} | closing {self.units} at price {price:.3f}')
        self.units = 0
        self.print_current_balance(bar)
        self.print_net_wealth(bar)
        print(f'{date} | net performance [%] = {perf:.3f}')
        print(f'{date} | trades = {self.trades}')
    

In [135]:
bb = BacktestBase('EUR=',10000)

In [136]:
bb.get_date_price(100)

('2010-05-27', 1.2368)

In [137]:
bb.print_net_wealth(100
                   
                   )

2010-05-27 | net wealth = 10000.00


In [138]:
bb.print_current_balance(200)

2010-10-19 | current balance = 10000.00


In [139]:
bb.place_buy_order(200,amount = 2500)

2010-10-19 | bought 1820 at price 1.3732


In [140]:
bb.print_net_wealth(210)

2010-11-02 | net wealth = 10055.51


In [141]:
bb.place_sell_order(500,units = 820)

2011-12-27 | sold 820 at price 1.3068


In [142]:
bb.print_current_balance(500)

2011-12-27 | current balance = 8572.35


In [143]:
bb.print_net_wealth(500)

2011-12-27 | net wealth = 9879.15


In [144]:
bb.close_out(1200)

2014-10-09 | closing 1000 at price 1.269
2014-10-09 | current balance = 9841.35
2014-10-09 | net wealth = 9841.35
2014-10-09 | net performance [%] = -0.016
2014-10-09 | trades = 2


### Event Based Backtest

In [183]:
class SMABacktester(BacktestBase):
    def prepare_statistics(self,SMA1,SMA2):
        self.data['SMA1'] = self.data[self.symbol].rolling(SMA1).mean()
        self.data['SMA2'] = self.data[self.symbol].rolling(SMA2).mean()
    def backtest_strategy(self,SMA1,SMA2):
        print(f'Backtesting SMA1={SMA1} | SMA2 = {SMA2}')
        print(55 * '=')
        self.prepare_statistics(SMA1,SMA2)
        self.units = 0
        self.trades = 0
        self.position = 0
        self.current_balance = self.initial_amount
        self.data['signal'] = np.where(self.data['SMA1']>self.data['SMA2'],1,-1)
        
        for bar in range(SMA2, len(self.data)):
            signal = self.data['signal'].iloc[bar]
            if self.position  in [0,-1] and signal==1:
                print(signal)
                self.place_buy_order(bar,units = (1-self.position)*1000)
                self.print_current_balance(bar)
                self.print_net_wealth(bar)
                self.position = 1
            elif self.position  in [0,1] and signal==-1:
                print(signal)
                self.place_sell_order(bar,units = (1+self.position)*1000)
                self.print_current_balance(bar)
                self.print_net_wealth(bar)
                self.position = -1
        self.close_out(bar)
        
        
        

In [184]:
sma = SMABacktester('EUR=',10000)

In [185]:
sma.backtest_strategy(30,60)

Backtesting SMA1=30 | SMA2 = 60
-1
2010-03-31 | sold 1000 at price 1.351
2010-03-31 | current balance = 11351.00
2010-03-31 | net wealth = 10000.00
1
2010-07-22 | bought 2000 at price 1.2888
2010-07-22 | current balance = 8773.40
2010-07-22 | net wealth = 10062.20
-1
2010-09-22 | sold 2000 at price 1.3399
2010-09-22 | current balance = 11453.20
2010-09-22 | net wealth = 10113.30
1
2010-09-28 | bought 2000 at price 1.3579
2010-09-28 | current balance = 8737.40
2010-09-28 | net wealth = 10095.30
-1
2010-12-07 | sold 2000 at price 1.3263
2010-12-07 | current balance = 11390.00
2010-12-07 | net wealth = 10063.70
1
2011-02-04 | bought 2000 at price 1.3585
2011-02-04 | current balance = 8673.00
2011-02-04 | net wealth = 10031.50
-1
2011-06-14 | sold 2000 at price 1.4438
2011-06-14 | current balance = 11560.60
2011-06-14 | net wealth = 10116.80
1
2011-08-29 | bought 2000 at price 1.4508
2011-08-29 | current balance = 8659.00
2011-08-29 | net wealth = 10109.80
-1
2011-09-09 | sold 2000 at pric